In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setup

In [ ]:
!pip install pytorch-crf transformers datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.6 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, CenteredNorm, PowerNorm
from torchcrf import CRF

from datasets import Dataset, DatasetDict, load_dataset, concatenate_datasets
import random
import seaborn as sn

import transformers
from transformers import BertTokenizerFast, DistilBertModel, DistilBertConfig
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup

from torch.optim import AdamW

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

## Dataset

In [ ]:
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        max_length=180, truncation=True, padding="max_length",
        is_split_into_words=True, add_special_tokens=False,
        return_tensors = "pt",
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)

        label = []
        for i in range(len(word_ids)):
          if word_ids[i] == None:
            label.append(0)
          elif labels[word_ids[i]] % 2 == 1 and word_ids[i-1] == word_ids[i]:
            label.append(labels[word_ids[i]] + 1)
          else:
            label.append(labels[word_ids[i]])
        new_labels.append(label)

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
label_to_tag = {
    'O':0,
    'B-Drug':1, 'I-Drug':2,
    'B-Reason':3, 'I-Reason':4,
    'B-Route':5, 'I-Route':6,
    'B-Strength':7, 'I-Strength':8,
    'B-Form':9, 'I-Form':10,
    'B-Dosage':11, 'I-Dosage':12,
    'B-Frequency':13, 'I-Frequency':14,
    'B-Duration':15, 'I-Duration':16,
    'B-ADE':17, 'I-ADE':18,
}

tag_to_label = {v:k for k,v in label_to_tag.items()}
LABELS = list(label_to_tag.keys())

nb_unique_tags = len(label_to_tag)

In [ ]:
def get_dataset(TOKENIZER_NAME, sam_BD):
  tokenizer = BertTokenizerFast.from_pretrained(TOKENIZER_NAME)

  data = {
    "train": {"tokens": [], "labels": [], "ner_tags": []},
    "valid": {"tokens": [], "labels": [], "ner_tags": []},
    "test": {"tokens": [], "labels": [], "ner_tags": []}
  }

  def populate_dataset(file_path, train_valid_or_test, sam_BD):
    with open(file_path) as f:
      tokens, labels, ner_tags = [], [], []
      for line in f.read().splitlines():
        if len(line) < 2:
          is_taken = (not sam_BD) or ner_tags.count(0) != len(ner_tags) or random.randint(1, 100) > 92
          if len(tokens) != 0 and is_taken: # bypass double line breaks
            data[train_valid_or_test]['tokens'].append(tokens)
            data[train_valid_or_test]['labels'].append(labels)
            data[train_valid_or_test]['ner_tags'].append(ner_tags)
          tokens, labels, ner_tags = [], [], []
          continue
        line = line.split(' ')
        token, label, tag = ' '.join(line[:-4]), line[-1], label_to_tag[line[-1]]
        tokens.append(token)
        labels.append(label)
        ner_tags.append(tag)


  populate_dataset(file_path='drive/MyDrive/TransformerCRF-v2/Dataset/train_spacy.txt', train_valid_or_test='train', sam_BD=sam_BD)
  populate_dataset(file_path='drive/MyDrive/TransformerCRF-v2/Dataset/valid_spacy.txt', train_valid_or_test='valid', sam_BD=sam_BD)
  populate_dataset(file_path='drive/MyDrive/TransformerCRF-v2/Dataset/test_spacy.txt', train_valid_or_test='test', sam_BD=False)

  train_dataset = Dataset.from_dict(data["train"])
  valid_dataset = Dataset.from_dict(data["valid"])
  test_dataset = Dataset.from_dict(data["test"])

  dataset = DatasetDict({
    "train": concatenate_datasets([train_dataset, valid_dataset]),
    "test": test_dataset
  })

  dataset = dataset.map(
      lambda x: tokenize_and_align_labels(x, tokenizer),
      batched=True,
      remove_columns=dataset["train"].column_names,
  ).with_format('torch')

  return dataset

14041 -> no-label: 2909 (20.72%)

41497 -> no-label: 31163 (75.10%) (2909 no-label reached at 3829th sample -> at this stage it represented 75.97%)

We want to only keep N no-label so we reduce to X% (we tos a coin with Y% chance to keep a no-label sample)

Now we get (on average):

## Model

In [ ]:
import os
from typing import List, Tuple

import torch
from torch import nn
from transformers import AutoModel

LOG_INF = 10e5

class BertCRF(nn.Module):
    def __init__(self, num_labels: int, use_crf, from_scratch):
        super().__init__()
        self.num_labels = num_labels
        self.use_crf = use_crf
        self.cross_entropy = nn.CrossEntropyLoss()

        # PLM
        if from_scratch:
            self.bert = DistilBertModel(config=DistilBertConfig(vocab_size=28996)) # len vocab of 'distilbert-base-cased'
        else:
            self.bert = AutoModel.from_pretrained(MODEL_NAME)

        # Classification/Adpation Layer
        self.dropout = nn.Dropout(0.2)
        self.hidden2label = nn.Linear(self.bert.config.hidden_size, num_labels)

        # CRF Layer
        if self.use_crf:
          self.start_transitions = nn.Parameter(torch.empty(num_labels))
          self.end_transitions = nn.Parameter(torch.empty(num_labels))
          self.transitions = nn.Parameter(torch.empty(num_labels, num_labels))

          nn.init.uniform_(self.start_transitions, -0.1, 0.1)
          nn.init.uniform_(self.end_transitions, -0.1, 0.1)
          nn.init.uniform_(self.transitions, -0.1, 0.1)

    def _compute_log_denominator(self, features: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        seq_len = features.shape[0]

        log_score_over_all_seq = self.start_transitions + features[0]

        for i in range(1, seq_len):
            next_log_score_over_all_seq = torch.logsumexp(
                log_score_over_all_seq.unsqueeze(2) + self.transitions + features[i].unsqueeze(1),
                dim=1,
            )
            log_score_over_all_seq = torch.where(
                mask[i].unsqueeze(1),
                next_log_score_over_all_seq,
                log_score_over_all_seq,
            )
        log_score_over_all_seq += self.end_transitions
        return torch.logsumexp(log_score_over_all_seq, dim=1)

    def _compute_log_numerator(self, features: torch.Tensor, labels: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        seq_len, bs, _ = features.shape

        score_over_seq = self.start_transitions[labels[0]] + features[0, torch.arange(bs), labels[0]]

        for i in range(1, seq_len):
            score_over_seq += (
                self.transitions[labels[i - 1], labels[i]] + features[i, torch.arange(bs), labels[i]]
            ) * mask[i]
        seq_lens = mask.sum(dim=0) - 1
        last_tags = labels[seq_lens.long(), torch.arange(bs)]
        score_over_seq += self.end_transitions[last_tags]
        return score_over_seq

    def get_bert_features(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        hidden = self.bert(input_ids, attention_mask=attention_mask)["last_hidden_state"]
        hidden = self.dropout(hidden)
        return self.hidden2label(hidden), hidden

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        features, _ = self.get_bert_features(input_ids=input_ids, attention_mask=attention_mask)
        attention_mask = attention_mask.bool()

        if self.use_crf:
            features = torch.swapaxes(features, 0, 1)
            attention_mask = torch.swapaxes(attention_mask, 0, 1)
            labels = torch.swapaxes(labels, 0, 1)

            log_numerator = self._compute_log_numerator(features=features, labels=labels, mask=attention_mask)
            log_denominator = self._compute_log_denominator(features=features, mask=attention_mask)

            return torch.mean(log_denominator - log_numerator)
        else:
            return self.cross_entropy(
                features.flatten(end_dim=1),
                torch.where(attention_mask.bool(), labels, -100).flatten(end_dim=1),
            )

    def _viterbi_decode(self, features: torch.Tensor, mask: torch.Tensor) -> List[List[int]]:
        seq_len, bs, _ = features.shape

        log_score_over_all_seq = self.start_transitions + features[0]

        backpointers = torch.empty_like(features)

        for i in range(1, seq_len):
            next_log_score_over_all_seq = (
                log_score_over_all_seq.unsqueeze(2) + self.transitions + features[i].unsqueeze(1)
            )

            next_log_score_over_all_seq, indices = next_log_score_over_all_seq.max(dim=1)

            log_score_over_all_seq = torch.where(
                mask[i].unsqueeze(1),
                next_log_score_over_all_seq,
                log_score_over_all_seq,
            )
            backpointers[i] = indices

        backpointers = backpointers[1:].int()

        log_score_over_all_seq += self.end_transitions
        seq_lens = mask.sum(dim=0) - 1

        best_paths = []
        for seq_ind in range(bs):
            best_label_id = torch.argmax(log_score_over_all_seq[seq_ind]).item()
            best_path = [best_label_id]

            for backpointer in reversed(backpointers[: seq_lens[seq_ind]]):
                best_path.append(backpointer[seq_ind][best_path[-1]].item())

            best_path.reverse()
            best_paths.append(best_path)

        return best_paths

    def decode(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> List[List[int]]:
        features, _ = self.get_bert_features(input_ids=input_ids, attention_mask=attention_mask)
        attention_mask = attention_mask.bool()

        if self.use_crf:
            features = torch.swapaxes(features, 0, 1)
            mask = torch.swapaxes(attention_mask, 0, 1)
            predictions = self._viterbi_decode(features=features, mask=mask)
            # Apply padding
            return torch.tensor([p + [0]*(len(attention_mask[-1]) - len(p)) for p in predictions])
        else:
            labels = torch.argmax(features, dim=2)
            predictions = []
            for i in range(len(labels)):
                predictions.append(labels[i][attention_mask[i]].tolist())
            # Apply padding
            return torch.tensor([p + [0]*(len(attention_mask[-1]) - len(p)) for p in predictions])

## Metrics

In [ ]:
def get_metrics(true_labels, predicted_labels, labels, binary):

  # Calculate accuracy, precision, recall, and F1-score for each class
  accuracy = accuracy_score(true_labels, predicted_labels)
  micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='micro', labels=labels)
  macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro', labels=labels)
  weight_precision, weight_recall, weight_f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted', labels=labels)

  # Classification reports
  metrics_report_bio = classification_report(true_labels, predicted_labels, labels=labels, output_dict=True)
  metrics_report_token = classification_report(np.array([i[2:] if i != 'O' else 'O' for i in true_labels]), np.array([i[2:] if i != 'O' else 'O' for i in predicted_labels]), labels=list(set([i[2:] if i != 'O' else 'O' for i in labels])), output_dict=True)
  metrics_report_text = classification_report(true_labels, predicted_labels, labels=labels, output_dict=False)
  if binary:
    metrics_report_binary = classification_report(np.array(['NAMED_ENTITY' if i != 'O' else 'O' for i in true_labels]), np.array(['NAMED_ENTITY' if i != 'O' else 'O' for i in predicted_labels]), output_dict=True)
  else:
    metrics_report_binary = None

  # Confusion matrix and Binary CF
  cf_bio = confusion_matrix(true_labels, predicted_labels, labels=labels)
  cf_token = confusion_matrix(np.array([i[2:] if i != 'O' else 'O' for i in true_labels]), np.array([i[2:] if i != 'O' else 'O' for i in predicted_labels]), labels=list(set([i[2:] if i != 'O' else 'O' for i in labels])))
  if binary:
    cf_binary = confusion_matrix(np.array(['NAMED_ENTITY' if i != 'O' else 'O' for i in true_labels]), np.array(['NAMED_ENTITY' if i != 'O' else 'O' for i in predicted_labels]))
  else:
    cf_binary = None

  return (
      labels,
      accuracy,
      (micro_precision, micro_recall, micro_f1),
      (macro_precision, macro_recall, macro_f1),
      (weight_precision, weight_recall, weight_f1),
      (metrics_report_bio, metrics_report_token, metrics_report_text, metrics_report_binary),
      (cf_bio, cf_token, cf_binary)
  )


In [ ]:
def calculate_metrics(predicted_labels, true_labels, masking):

  predicted_labels = predicted_labels.cpu().numpy()
  true_labels = true_labels.cpu().numpy()
  masking = masking.cpu().numpy()

  # Flatten the arrays to 1D arrays
  predicted_labels_flat = predicted_labels.flatten()
  true_labels_flat = true_labels.flatten()
  masking = masking.flatten()

  # Use valid indices to filter out -100 tokens
  predicted_labels_flat = predicted_labels_flat[masking]
  true_labels_flat = true_labels_flat[masking]

  # Replace tags with labels
  predicted_labels_flat = np.array([tag_to_label[tag] for tag in predicted_labels_flat])
  true_labels_flat = np.array([tag_to_label[tag] for tag in true_labels_flat])

  # ----------------------------------------------------
  RESULTS = {}

  # Metrics (normal/everything)
  RESULTS['default'] = get_metrics(true_labels_flat, predicted_labels_flat, labels=LABELS, binary=True)

  # Metrics without O
  RESULTS['no_O'] = get_metrics(true_labels_flat, predicted_labels_flat, labels=[l for l in LABELS if l != 'O'], binary=False)

  # Metrics without ADE/Reason
  RESULTS['no_ADE_REASON'] = get_metrics(true_labels_flat, predicted_labels_flat, labels=[l for l in LABELS if l not in ['B-ADE', 'I-ADE', 'B-Reason', 'I-Reason']], binary=True)

  # Metrics without O/ADE/Reason
  RESULTS['no_O_ADE_REASON'] = get_metrics(true_labels_flat, predicted_labels_flat, labels=[l for l in LABELS if l not in ['O', 'B-ADE', 'I-ADE', 'B-Reason', 'I-Reason']], binary=False)

  # Metrics with ADE replaced by Reason
  new_true_labels_flat = np.where(true_labels_flat == 'B-ADE', 'B-Reason', true_labels_flat)
  new_true_labels_flat = np.where(new_true_labels_flat == 'I-ADE', 'I-Reason', new_true_labels_flat)

  new_predicted_labels_flat = np.where(predicted_labels_flat == 'B-ADE', 'B-Reason', predicted_labels_flat)
  new_predicted_labels_flat = np.where(new_predicted_labels_flat == 'I-ADE', 'I-Reason', new_predicted_labels_flat)

  RESULTS['with_ADE_as_REASON'] = get_metrics(new_true_labels_flat, new_predicted_labels_flat, labels=[l for l in LABELS if l not in ['B-ADE', 'I-ADE']], binary=True)

  return RESULTS

## Train function

In [ ]:
def train(model, train_dataloader, eval_dataloader, nb_epochs, optimizer, scheduler, device, RUN_NAME):

  for epoch in range(nb_epochs):
    print("Epoch", epoch+1, end=" - ")

    # Training
    model.train()
    epoch_loss = []
    for batch in train_dataloader:

      optimizer.zero_grad()

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

      loss.backward()
      epoch_loss.append(loss.item())

      optimizer.step()
      scheduler.step()

    training_loss = np.mean(epoch_loss)
    print("Training Loss:", training_loss, end=" - ")

    # Evaluation
    model.eval()
    epoch_loss = []
    all_predictions, all_labels, all_masks = torch.tensor([], device=device), torch.tensor([], device=device), torch.tensor([], device=device)
    for batch in eval_dataloader:

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      with torch.no_grad():
        predictions = model.decode(input_ids=input_ids, attention_mask=attention_mask)
        loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        epoch_loss.append(loss.item())

      all_predictions = torch.cat((all_predictions, predictions.to(device)), dim=0)
      all_labels = torch.cat((all_labels, labels.to(device)), dim=0)
      all_masks = torch.cat((all_masks, attention_mask), dim=0)

    eval_loss = np.mean(epoch_loss)
    print("Evaluation Loss:", eval_loss)

  return calculate_metrics(all_predictions, all_labels, all_masks.bool())

## Hyperparameter selection pipeline

In [2]:
hyperparameters = [
    # Model_name, Batch_size, Epochs, Learning rate, Uses CRF Layer (bool), Uses Sam_BD (bool)
    ('bert-base-cased', 14, 3, 5e-5, True, True),
    ('bert-base-cased', 14, 3, 5e-5, True, False),

    ('bert-base-cased', 8, 3, 5e-5, False, True),
    ('bert-base-cased', 8, 3, 5e-5, False, False),

    ('dmis-lab/biobert-base-cased-v1.2', 8, 2, 5e-5, True, True),
    ('dmis-lab/biobert-base-cased-v1.2', 8, 2, 5e-5, True, False),

    ('dmis-lab/biobert-base-cased-v1.2', 8, 2, 1e-4, False, True),
    ('dmis-lab/biobert-base-cased-v1.2', 8, 2, 1e-4, False, False),

    ('medicalai/ClinicalBERT', 14, 2, 5e-5, True, True),
    ('medicalai/ClinicalBERT', 14, 2, 5e-5, True, False),

    ('medicalai/ClinicalBERT', 14, 1, 1e-4, False, True),
    ('medicalai/ClinicalBERT', 14, 1, 1e-4, False, False),

    ('TransformerCRF-distilbert-base-cased', 8, 4, 1e-4, True, True),
    ('TransformerCRF-distilbert-base-cased', 8, 4, 1e-4, True, False),

    ('TransformerCRF-distilbert-base-cased', 14, 4, 1e-4, False, True),
    ('TransformerCRF-distilbert-base-cased', 14, 4, 1e-4, False, False),
]

In [ ]:
datasets = {
    'bert-base-cased-True': get_dataset(TOKENIZER_NAME = 'bert-base-cased', sam_BD=True),
    'dmis-lab/biobert-base-cased-v1.2-True': get_dataset(TOKENIZER_NAME = 'dmis-lab/biobert-base-cased-v1.2', sam_BD=True),
    'medicalai/ClinicalBERT-True': get_dataset(TOKENIZER_NAME = 'medicalai/ClinicalBERT', sam_BD=True),
    'TransformerCRF-distilbert-base-cased-True': get_dataset(TOKENIZER_NAME = 'distilbert-base-cased', sam_BD=True),

    'bert-base-cased-False': get_dataset(TOKENIZER_NAME = 'bert-base-cased', sam_BD=False),
    'dmis-lab/biobert-base-cased-v1.2-False': get_dataset(TOKENIZER_NAME = 'dmis-lab/biobert-base-cased-v1.2', sam_BD=False),
    'medicalai/ClinicalBERT-False': get_dataset(TOKENIZER_NAME = 'medicalai/ClinicalBERT', sam_BD=False),
    'TransformerCRF-distilbert-base-cased-False': get_dataset(TOKENIZER_NAME = 'distilbert-base-cased', sam_BD=False),
}

In [ ]:
RESULTS = {}

for hyperparam in hyperparameters:
  MODEL_NAME, BATCH_SIZE, NUM_EPOCHS, LEARNING_RATE, USES_CRF, USES_SAMBD = hyperparam
  RUN_NAME = f"{MODEL_NAME.replace('/','_')} - {BATCH_SIZE} - {LEARNING_RATE} - {USES_CRF} - {USES_SAMBD}"

  dataset = datasets[f'{MODEL_NAME}-{USES_SAMBD}']

  train_dataloader = DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True)
  eval_dataloader = DataLoader(dataset['test'], batch_size=BATCH_SIZE, shuffle=False)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  model = BertCRF(num_labels=nb_unique_tags, use_crf=USES_CRF, from_scratch=('Transformer' in MODEL_NAME)).to(device)

  optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
  total_steps = len(train_dataloader) * NUM_EPOCHS
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

  # Train
  print('\nModel:', RUN_NAME)
  RESULTS[RUN_NAME] = train(model, train_dataloader, eval_dataloader, NUM_EPOCHS, optimizer, scheduler, device, RUN_NAME)

  # Save model
  if not os.path.exists(f"drive/MyDrive/TransformerCRF-v2/Results/{RUN_NAME.replace('/', '_')}"):
      os.makedirs(f"drive/MyDrive/TransformerCRF-v2/Results/{RUN_NAME.replace('/', '_')}")
  torch.save(model, f"drive/MyDrive/TransformerCRF-v2/Results/{RUN_NAME.replace('/', '_')}/saved_model")

## Save model results

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib

def plot_classification_report(clf_report, title, figsize, dpi, save_fig_path=None, **kwargs):
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)

    keys_to_plot = [key for key in clf_report.keys()]
    if 'binary' in title: keys_to_plot = [key for key in clf_report.keys() if key != 'accuracy']
    df = pd.DataFrame(clf_report, columns=keys_to_plot).T
    #the following line ensures that dataframe are sorted from the majority classes to the minority classes
    df.sort_values(by=['support'], inplace=True)

    #first, let's plot the heatmap by masking the 'support' column
    rows, cols = df.shape
    mask = np.zeros(df.shape)
    mask[:,cols-1] = True

    ax = sns.heatmap(df, mask=mask, annot=True, cmap=plt.cm.Blues, fmt='.3g',
            vmin=0.0,
            vmax=1.0,
            linewidths=1, linecolor='black'
                    )

    #then, let's add the support column by normalizing the colors in this column
    mask = np.zeros(df.shape)
    mask[:,:cols-1] = True

    ax = sns.heatmap(df, mask=mask, annot=True, cmap=plt.cm.Blues, cbar=False,
            linewidths=1, linecolor='black', fmt='.0f',
            vmin=df['support'].min(),
            vmax=df['support'].sum(),
            norm=mpl.colors.Normalize(vmin=df['support'].min(),
                                      vmax=df['support'].sum())
                    )

    plt.title(title)
    plt.xticks(rotation = 45)
    plt.yticks(rotation = 360)

    if (save_fig_path != None):
        path = pathlib.Path(save_fig_path)
        path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(save_fig_path)

In [ ]:
for MODEL_NAME, results in RESULTS.items():
  for RES_TYPE, res in results.items():
    FILE_PATH = f"drive/MyDrive/TransformerCRF-v2/Results/{MODEL_NAME.replace('/', '_')}/{RES_TYPE}"
    print(FILE_PATH, end=' - ')
    labels, accuracy, (micro_precision, micro_recall, micro_f1), (macro_precision, macro_recall, macro_f1), (weight_precision, weight_recall, weight_f1), (metrics_report_bio, metrics_report_token, metrics_report_text, metrics_report_binary), (cf_bio, cf_token, cf_binary)= res

    if not os.path.exists(FILE_PATH):
      os.makedirs(FILE_PATH)

    # Save overall metric scores
    with open(f"{FILE_PATH}/scores.txt", 'w') as f:
      f.write(f"Accuracy: {accuracy}\n")
      f.write(f"Macro Scores (p/r/f1): {macro_precision}/{macro_recall}/{macro_f1}\n")
      f.write(f"Micro Scores (p/r/f1): {micro_precision}/{micro_recall}/{micro_f1}\n")
      f.write(f"Weighted Scores (p/r/f1): {weight_precision}/{weight_recall}/{weight_f1}\n")

    # Save Classification Reports (bio, token and binary)
    with open(f"{FILE_PATH}/bio_classification_report.txt", 'w') as f:
      f.write(metrics_report_text)

    plot_classification_report(
      metrics_report_bio,
      title='Classification Report on the Testing Set (BIO evaluation)\n',
      figsize=(10, 10), dpi=100,
      save_fig_path = f"{FILE_PATH}/bio_classification_report.png"
    )

    plot_classification_report(
      metrics_report_token,
      title='Classification Report on the Testing Set (Token evaluation)\n',
      figsize=(10, 10), dpi=100,
      save_fig_path = f"{FILE_PATH}/token_classification_report.png"
    )

    if metrics_report_binary is not None:
      plot_classification_report(
        metrics_report_binary,
        title='Classification Report on the Testing Set (Binary evaluation)\n',
        figsize=(10, 10), dpi=100,
        save_fig_path = f"{FILE_PATH}/binary_classification_report.png"
      )

    # Save Confusion Matrix (bio, token and binary)
    df_cm = pd.DataFrame(cf_bio, index=labels, columns=labels)
    plt.figure(figsize = (12,12))
    plt.title("Confusion Matrix on the Testing Set (BIO evaluation)\n")
    try: sn.heatmap(df_cm, annot=True, cmap=plt.cm.Blues, fmt='.0f', norm=LogNorm())
    except:
      print('FAILED')
      continue
    plt.xlabel("\nPredicted Labels")
    plt.ylabel("True Labels\n")
    plt.savefig(f"{FILE_PATH}/bio_confusion_matrix.png")

    df_cm = pd.DataFrame(cf_token, index=list(set([i[2:] if i != 'O' else 'O' for i in labels])), columns=list(set([i[2:] if i != 'O' else 'O' for i in labels])))
    plt.figure(figsize = (12,12))
    plt.title("Confusion Matrix on the Testing Set (Token evaluation)\n")
    try: sn.heatmap(df_cm, annot=True, cmap=plt.cm.Blues, fmt='.0f', norm=LogNorm())
    except:
      print('FAILED')
      continue
    plt.xlabel("\nPredicted Labels")
    plt.ylabel("True Labels\n")
    plt.savefig(f"{FILE_PATH}/token_confusion_matrix.png")

    if cf_binary is not None:
      df_cm = pd.DataFrame(cf_binary, index=['NAMED_ENTITY', 'O'], columns=['NAMED_ENTITY', 'O'])
      plt.figure(figsize = (10,10))
      plt.title("Confusion Matrix on the Testing Set (Binary evaluation)\n")
      try: sn.heatmap(df_cm, annot=True, cmap=plt.cm.Blues, fmt='.0f', norm=LogNorm())
      except:
        print('FAILED')
        continue
      plt.xlabel("\nPredicted Labels")
      plt.ylabel("True Labels\n")
      plt.savefig(f"{FILE_PATH}/binary_confusion_matrix.png")

    print('SUCCESS')